<a href="https://colab.research.google.com/github/david-1192/Senales_y_Sistemas_2025/blob/main/Dashboards_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Streamlit | Cristian David Chalaca Salas**

In [79]:
#instalación de librerías
!pip install streamlit -q
# !pip install numpy matplotlib scipy pandas

In [80]:
#instalar librerias necesarias para descargar audios youtube
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz -q
#Libreria para manipulacion de archivos de audio
!pip install soundfile -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [81]:
!pip install browser-cookie3

In [82]:
!apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [83]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [84]:
%%writefile 0_Home.py

import streamlit as st

st.set_page_config(
    page_title="PowerAMP",
    page_icon="👽​",
)

st.title("👽 PowerAMP")

st.markdown(
    """
    ### Aplicaciones disponibles en este Dashboard:

    1. 📻 **Aplicación en comunicaciones --> Modulación AM**.
    2. 🚀 **Aplicación en circuitos eléctricos --> Potencia**

    ---
    Selecciona una página de alguna aplicación desde el menú lateral izquierdo para comenzar.
    """
)

Overwriting 0_Home.py


##Dashboard para la aplicación de Modulción AM:

In [85]:
%%writefile pages/1_🔊_AM_Modulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from pydub import AudioSegment
import yt_dlp
import os
from io import BytesIO

st.set_page_config(page_title="🎵 AM Modulation Explorer", layout="wide")
st.title("🔊 Modulación AM de tu canción favorita")

url = st.text_input("🎧 Ingresa el enlace de YouTube de tu canción favorita:", "")

# Funciones auxiliares
def array_to_audiosegment(signal, sample_rate):
    signal_int16 = np.int16(signal / np.max(np.abs(signal)) * 32767)
    audio = AudioSegment(
        signal_int16.tobytes(),
        frame_rate=sample_rate,
        sample_width=2,
        channels=1
    )
    return audio

def export_audiosegment_to_bytes(audiosegment):
    buf = BytesIO()
    audiosegment.export(buf, format="wav")
    return buf.getvalue()

if url:
    try:
        with st.spinner("🔍 Descargando y procesando audio... Esto puede tardar unos segundos"):
            # Descargar audio
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': 'audio.%(ext)s',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                }],
                'quiet': True,
                'no_warnings': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            audio = AudioSegment.from_file("audio.mp3").set_channels(1).set_frame_rate(44100)
            fragment = audio[20000:25000]  # 5 segundos
            os.remove("audio.mp3")

            # Convertir a numpy
            samples = np.array(fragment.get_array_of_samples()).astype(np.float32) / 2**15
            Fs = fragment.frame_rate
            t = np.linspace(0, len(samples)/Fs, len(samples))

            # Señal mensaje
            m_t = samples

            # Parámetros portadora
            Ac = 1.0
            Fc = 10000  # Hz

            # Portadora
            c_t = Ac * np.cos(2 * np.pi * Fc * t)

            # Modulación AM
            y_t = (1 + m_t) * c_t

            # Mezcla (señal modulada * portadora)
            mixed = y_t * c_t

            # FFT de la señal mezclada
            M = fft(mixed)
            f = fftfreq(len(mixed), 1/Fs)
            M_filtered = M.copy()
            M_filtered[np.abs(f) > 8000] = 0  # Filtro ideal
            demod = np.real(ifft(M_filtered))

        st.success("✅ Audio descargado y procesado correctamente!")

        # Conversión a audio
        audio_m = fragment
        audio_portadora = array_to_audiosegment(c_t, Fs)
        audio_modulada = array_to_audiosegment(y_t, Fs)
        audio_demodulada = array_to_audiosegment(demod, Fs)

        # Reproducción
        st.subheader("🎼 Escucha y compara las señales")
        st.audio(export_audiosegment_to_bytes(audio_m), format="audio/wav")
        st.caption("📢 *Señal Mensaje Original*")

        st.audio(export_audiosegment_to_bytes(audio_portadora), format="audio/wav")
        st.caption("🎶 *Señal Portadora*")

        st.audio(export_audiosegment_to_bytes(audio_modulada), format="audio/wav")
        st.caption("📡 *Señal Modulada AM*")

        st.audio(export_audiosegment_to_bytes(audio_demodulada), format="audio/wav")
        st.caption("🔄 *Señal Demodulada (filtrada)*")

        # Gráficas en el tiempo
        st.subheader("🕒 Gráficas en el tiempo")
        fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
        axs[0].plot(t, m_t, color='navy'); axs[0].set_title("Señal Mensaje m(t)"); axs[0].grid()
        axs[1].plot(t, c_t, color='gray'); axs[1].set_title("Señal Portadora c(t)"); axs[1].grid()
        axs[2].plot(t, y_t, color='orange'); axs[2].set_title("Señal Modulada y(t)"); axs[2].grid()
        axs[3].plot(t, mixed, color='purple'); axs[3].set_title("Señal Mezclada y(t) * c(t)"); axs[3].set_xlabel("Tiempo (s)"); axs[3].grid()
        st.pyplot(fig)

        # Espectros
        st.subheader("🌐 Espectros de frecuencia")
        def plot_spectrum(signal, Fs, title):
            N = len(signal)
            spectrum = fft(signal)
            freqs = fftfreq(N, 1/Fs)
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.plot(freqs[:N//2], np.abs(spectrum[:N//2]), color='darkred')
            ax.set_title(title)
            ax.set_xlabel("Frecuencia (Hz)")
            ax.set_ylabel("Magnitud")
            ax.grid()
            return fig

        st.pyplot(plot_spectrum(m_t, Fs, "Espectro de la Señal Mensaje"))
        st.pyplot(plot_spectrum(c_t, Fs, "Espectro de la Portadora"))
        st.pyplot(plot_spectrum(y_t, Fs, "Espectro de la Señal Modulada"))
        st.pyplot(plot_spectrum(mixed, Fs, "Espectro de la Señal Mezclada"))
        st.pyplot(plot_spectrum(demod, Fs, "Espectro de la Señal Demodulada"))

        # Comparación final
        st.subheader("🔁 Comparación final entre señal original y demodulada")
        fig2, ax2 = plt.subplots(figsize=(10, 4))
        ax2.plot(t, m_t, label="Original", color='navy')
        ax2.plot(t, demod, label="Demodulada", color='green', alpha=0.7)
        ax2.legend()
        ax2.set_xlabel("Tiempo (s)")
        ax2.set_title("Comparación señal original vs demodulada")
        ax2.grid()
        st.pyplot(fig2)

        st.markdown("""
        ---
        🎉 *¡Listo!* Has visualizado todo el proceso de modulación y demodulación AM, incluyendo señales y espectros.
        ¡Prueba con otros enlaces y sigue explorando el mundo del procesamiento de señales!
        """)

    except Exception as e:
        st.error(f"⚠ Ocurrió un error: {e}")
else:
    st.info("⏳ Ingresa una URL para comenzar la exploración musical con modulación AM.")

Overwriting pages/1_🔊_AM_Modulacion.py


### Dashboard para la aplicación de circuitos eléctricos y potencia

In [86]:
%%writefile pages/2_🔌_Rectificador_RC.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.fft import rfft, rfftfreq

# Configuración general del dashboard
st.set_page_config(page_title="Rectificador RC - Análisis", layout="wide")
st.title("🔌 Rectificador de Onda Completa con Carga RC")

# Sidebar: Parámetros configurables por el usuario
with st.sidebar:
    st.header("Configuración del Circuito")
    R = st.slider("Resistencia R en ohmios (Ω)", 10, 5000, 100)
    C = st.slider("Capacitancia C en microfaradios (μF)", 1, 1000, 100)
    usar_log = st.checkbox("Usar escala logarítmica en espectro", value=False)

# Parámetros base de simulación
Fo = 60                       # Frecuencia de entrada [Hz]
Fs = 30 * Fo                  # Frecuencia de muestreo
To = 1 / Fo                   # Periodo de entrada
Ts = 1 / Fs                   # Periodo de muestreo
t = np.arange(0, 5 * To, Ts)  # Simulación de 5 ciclos
A = 170                       # Amplitud pico (equivale a 120Vrms)

# Señal sinusoidal y rectificación ideal
in_o = A * np.sin(2 * np.pi * Fo * t)
in_rc = np.abs(in_o)  # Rectificación de onda completa

# Función de transferencia del circuito RC
RC = R * C * 1e-6  # Convertir C a Faradios
num = [1]
den = [RC, 1]
system = sig.TransferFunction(num, den)
tout, out, _ = system.output(U=in_rc, T=t, X0=[0])  # Simulación de respuesta

# FFT de la señal de salida
Xf = rfft(out)
vfre = rfftfreq(len(out), Ts)
magnitude = 2 * np.abs(Xf) / len(out)  # Magnitud normalizada (ajustada)

# Cálculo de THD y FP
f0 = 60
fundamental_idx = np.argmin(np.abs(vfre - f0))
V1 = magnitude[fundamental_idx]

harmonics_sum = 0
for k in range(2, int(np.max(vfre) // f0) + 1):
    harmonic_freq = k * f0
    idx = np.argmin(np.abs(vfre - harmonic_freq))
    if idx < len(magnitude):
        Vk = magnitude[idx]
        harmonics_sum += Vk**2

THD = np.sqrt(harmonics_sum) / V1 if V1 > 0 else 0
PF = 1 / np.sqrt(1 + THD**2) if THD != 0 else 1

# ------------------------
# Visualización de resultados
# ------------------------

col1, col2 = st.columns(2)

with col1:
    st.subheader("Formas de Onda")
    fig, ax = plt.subplots(3, 1, figsize=(10, 8))
    ax[0].plot(t, in_o)
    ax[0].set_title("Voltaje de Entrada (Sinusoidal)")
    ax[1].plot(t, in_rc)
    ax[1].set_title("Señal Rectificada (Ideal)")
    ax[2].plot(tout, out)
    ax[2].set_title("Salida del Circuito RC")
    for a in ax:
        a.grid(True)
    plt.tight_layout()
    st.pyplot(fig)

with col2:
    st.subheader("Espectro de Frecuencias")
    fig2, ax2 = plt.subplots(figsize=(10, 4))

    # Filtrar solo frecuencias mayores que 0 para evitar log(0)
    valid_idx = vfre > 0
    vfre_valid = vfre[valid_idx]
    magnitude_valid = magnitude[valid_idx]

    ax2.stem(vfre_valid, magnitude_valid, basefmt=" ", linefmt='b-', markerfmt='bo')
    ax2.axvline(vfre[fundamental_idx], color='red', linestyle='--', label='Fundamental (60 Hz)')
    ax2.set_xlabel("Frecuencia (Hz)")
    ax2.set_ylabel("Magnitud")
    ax2.set_title("Espectro de la Salida")

    # Aplicar escalas si se selecciona logarítmico
    if usar_log:
        ax2.set_xscale("log")
        ax2.set_yscale("log")

    ax2.legend()
    ax2.grid(True, which='both')
    st.pyplot(fig2)


    st.subheader("Resultados")
    st.metric(label="THD", value=f"{THD * 100:.2f}%")
    st.metric(label="Factor de Potencia (estimado)", value=f"{PF:.4f}")

    st.info(f"Frecuencia fundamental detectada: {vfre[fundamental_idx]:.1f} Hz")

# Notas al pie
st.markdown("""
### 📝 Notas:
- Un **THD alto** indica presencia significativa de armónicos no deseados.
- Un **factor de potencia cercano a 1** indica un mejor aprovechamiento de la energía.
- La **frecuencia fundamental** en este análisis se espera alrededor de 60 Hz.
""")

Writing pages/2_🔌_Rectificador_RC.py


In [87]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_Home.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-07 04:21:57--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.10.1/cloudflared-linux-amd64 [following]
--2025-11-07 04:21:57--  https://github.com/cloudflare/cloudflared/releases/download/2025.10.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/d78430f2-7684-4130-ab31-f00b6b0d15e8?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-07T05%3A07%3A47Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-07

In [88]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
